# Introduction

In `main.ipynb` the goodfire API is called to collect experimental data into various CSVs. This code processes and plots that raw data.

In [ ]:
import pandas as pd

# Load the data
#df = pd.read_csv("data/20241123205432.csv")
df = pd.read_csv("data/20241124001836.csv")
df[:3]


In [ ]:
# Filter the data
df = df[(-0.8 < df['steerage']) & (df['steerage'] < 0.8)]

# Summarize the scores from the questions as two factors. 
# The Oxford Utilitarianism Scale has odd questions as coding for "Impartial Beneficence" and even as "Instrumental Harm",
# each of which is a simple average of responses.
df['factor'] = df['question'].apply(lambda x: 'IB' if x%2 == 0 else 'IH')
df2 = df[['feature', 'steerage', 'factor', 'mean_score']].groupby(['feature', 'steerage', 'factor'], as_index=False).mean()
df2 = df2.pivot(index=['feature', 'steerage'],columns='factor', values='mean_score').reset_index()
df2[:3]



In [ ]:

# Load and process the human dataset. TODO: Share code with pandelis for filtering
h1 = pd.read_csv("ous_data/data_preprocessed.csv")


h2 = h1.copy()
h2["IB"] = (h2["IB1"] + h2["IB2"] + h2["IB3"] + h2["IB4"] + h2["IB5"]) / 5
h2["IH"] = (h2["IH1"] + h2["IH2"] + h2["IH3"] + h2["IH4"]) / 5

h2[["sex", "country3", "Age", "IB", "IH"]][:3]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from scipy.stats import gaussian_kde


def scatter_plot_feature(feature, group):
    """Group should have IB/IH columns and be in sorted order"""
    # Draw the actual dots
    sc = plt.scatter(group['IB'], group['IH'],  marker='o',label=feature)
    color = sc.get_facecolor()
    # Connect the dots. These are draw as arrows so it's clear which end of series
    # is positive.
    for i in range(len(group) - 1):
        plt.annotate(
            '',  # No text for the annotation
            xy=(group['IB'].iloc[i + 1], group['IH'].iloc[i + 1]),  # Arrow end
            xytext=(group['IB'].iloc[i], group['IH'].iloc[i]),  # Arrow start
            arrowprops=dict(arrowstyle='->', lw=1.5, color=color),  # Arrow style
        )
    # Also label the chart
    plt.text(group['IB'].iloc[-1] + 0.04, group['IH'].iloc[-1], feature, fontsize=9, ha='left', va='center')

def human_heatmap():
    # Draw humans a heatmap
    ib_vals = np.arange(1, 8)
    ih_vals = np.arange(1, 8)
    heatmap, xedges, yedges = np.histogram2d(
        h2['IB'], h2['IH'], bins=(ib_vals, ih_vals)
    )
    plt.pcolormesh(xedges, yedges, heatmap.T, cmap='viridis', shading='auto', alpha=0.3)

def human_kde():
    # TODO: Not sure this is correct?
    # Draw humans as KDE
    smoothness=20
    ib_vals = np.linspace(1, 7, smoothness)  # Smoother grid for IB
    ih_vals = np.linspace(1, 7, smoothness)  # Smoother grid for IH
    ib_grid, ih_grid = np.meshgrid(ib_vals, ih_vals)
    positions = np.vstack([ib_grid.ravel(), ih_grid.ravel()])
    values = np.vstack([h2['IB'], h2['IH']])
    kernel = gaussian_kde(values)
    Z = np.reshape(kernel(positions).T, ib_vals.shape + ih_vals.shape)

    #plt.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r, extent=[1, 7, 1, 7])
    plt.imshow(np.rot90(np.rot90(np.rot90(Z))), cmap=plt.cm.gist_earth_r, extent=[1, 7, 1, 7])

def make_plot(df2):

    plt.figure(figsize=(10, 6))

    #human_heatmap()
    #human_kde()

    for feature, group in df2.groupby('feature'):
        scatter_plot_feature(feature, group)


    # Add labels, legend, and grid
    # plt.xlim(1, 7)
    # plt.ylim(1, 7)
    # plt.xticks(range(1, 8))  # Tick marks from 1 to 7
    # plt.yticks(range(1, 8))  # Tick marks from 1 to 7
    plt.xlabel('IB')
    plt.ylabel('IH')
    plt.title('Feature steerage effect on Instrumental Harm and Impartial Beneficence')
    plt.grid(True)

    # Show plot
    plt.show()

def chunks(xs, n):
    n = max(1, n)
    return (xs[i:i+n] for i in range(0, len(xs), n))


# TODO: Sort features by shape?
all_features = df2["feature"].unique()
for features in chunks(all_features, 5):
    make_plot(df2[df2["feature"].isin(features)])